In [ ]:
!pip install tensorflow pillow opencv-python
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('training set',
                                                 target_size=(64, 64),
                                                 batch_size=7,
                                                 class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('test-set',
                                            target_size=(64, 64),
                                            batch_size=7,
                                            class_mode='categorical')


In [ ]:
# Building the CNN
cnn = tf.keras.models.Sequential()

# Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# Full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer - Updated for 10 classes
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))

# Compiling CNN
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training and evaluating the CNN
cnn.fit(x=training_set, validation_data=test_set, epochs=8)

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:

cap = cv2.VideoCapture(0) 

In [ ]:
class_labels = list(training_set.class_indices.keys())


confidence_threshold = 0.5

In [3]:

confidence_threshold = 0.5

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Convert frame to grayscale for face detection
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract the face region
        face = frame[y:y+h, x:x+w]
        
        # Preprocess the face image for CNN prediction
        face_img = cv2.resize(face, (64, 64))
        face_img = image.img_to_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        face_img = face_img / 255.0  # Normalize the image
        
        # Predict using the trained CNN model
        result = cnn.predict(face_img)
        predicted_class_index = np.argmax(result)
        prediction_confidence = np.max(result)  # Get the highest confidence score
        
        # Check if the confidence is above the threshold
        if prediction_confidence >= confidence_threshold:
            prediction = class_labels[predicted_class_index]
        else:
            prediction = "New face detected"
        
        # Draw rectangle around the face and display the predicted name or message
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the frame with face detection and predictions
    cv2.imshow('Face Recognition', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


1/1 [==============================] - 0s 17ms/step


In [ ]:
# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()